In [1]:

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries import complete.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [2]:
#get data
request = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
my_table  = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]
my_table.rename(columns = {'Postcode':'PostalCode'},inplace=True)

In [3]:
# process assign borough
my_table.set_index('Borough').drop('Not assigned').reset_index(inplace=True)

In [4]:
my_table.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
my_table = my_table.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [6]:
# above . More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [8]:
my_table.loc[my_table['Neighbourhood']=='Not assigned','Neighbourhood'] = my_table.loc[my_table['Neighbourhood']=='Not assigned']['Borough']

In [9]:
my_table.loc[my_table['PostalCode']=='M5A']

,PostalCode,Borough,Neighbourhood
80,M5A,Downtown Toronto,Harbourfront


In [10]:
my_table.loc[my_table['Neighbourhood']=="Queen's Park"]

,PostalCode,Borough,Neighbourhood
120,M7A,Queen's Park,Queen's Park
160,M9A,Queen's Park,Queen's Park


In [11]:
my_table.shape

(180, 3)

In [14]:
#reading the file to coord dataframe
my_table['Latitude'] = '0';
my_table['Longitude'] = '0';

coord = pd.read_csv('https://cocl.us/Geospatial_data')


In [15]:
#merging dataframe that contain coordinates with the one that contains borough names
for i in my_table.index:
    for j in coord.index:
        if my_table.iloc[i, 0] == coord.iloc[j, 0]:
            my_table.iloc[i, 3] = coord.iloc[j, 1]
            my_table.iloc[i, 4] = coord.iloc[j, 2]

#checking the results            
my_table.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,0,0
1,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
4,M1G,Scarborough,Woburn,43.771,-79.2169


In [16]:
toronto = my_table[my_table['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.6796,-79.3775
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.668,-79.3677
2,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
3,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3789


In [17]:
#coordinates for Toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.
